一樣先讀資料

In [1]:
import pandas as pd
import numpy as np

In [2]:
college = pd.read_excel('學院科系V1.xlsx')

In [3]:
college.head()

,院,系所
0,工學院,土木工程
1,工學院,土木工程學
2,工學院,土木與防災碩、博士班
3,工學院,工工
4,工學院,工程科學


讀檔案並寫入專業度

In [4]:
data_manager = pd.read_excel('2-1.董監經理人學經歷_金融業_2017.xlsx')
data_manager = data_manager.drop(['資料源年月', '資料源', '資料截止日', '初任日期--董監事', '身份代碼', '獨立代碼', '會計主管', '財務主管', '重整代碼', '沿用日', '財務', '會計', '法務'], axis=1)

In [5]:
data_man = data_manager.groupby(['姓名代碼'])

In [6]:
df_final = pd.DataFrame()

#keyword = ['大學', '學校', '系', '專科', '學院', '商專'] 
#可以辨別是否是學歷的關鍵字，有出現這些字則高度可能是學歷

for name, group in data_man:
    dftemp = group.drop(['行次', '學經歷及目前兼任說明'], axis=1).iloc[0]
#     print(dftemp)
    dftemp['學歷'] = ''
    dftemp['經歷'] = ''
    dftemp['目前兼任'] = ''
#     print(dftemp)
    for index, row in group.iterrows(): #偵測關鍵字，含有則填入學歷的欄位，若是教授則填入經歷
        #(這裡原本想用"學"當作關鍵字，結果發現有職位叫"學習型組織主管"，只好改用複雜的方式處理
#         print(row['行次'])
        
        if '大學' in row['學經歷及目前兼任說明']:
            if '教授' in row['學經歷及目前兼任說明']:
                dftemp['經歷'] += row['學經歷及目前兼任說明'] + '、'
            else:
                dftemp['學歷'] += row['學經歷及目前兼任說明'] 
        elif '學院' in row['學經歷及目前兼任說明']:
            if '教授' in row['學經歷及目前兼任說明']:
                dftemp['經歷'] += row['學經歷及目前兼任說明'] + '、'
            else:
                dftemp['學歷'] += row['學經歷及目前兼任說明']
        elif '專科' in row['學經歷及目前兼任說明']:
            if '教授' in row['學經歷及目前兼任說明']:
                dftemp['經歷'] += row['學經歷及目前兼任說明'] + '、'
            else:
                dftemp['學歷'] += row['學經歷及目前兼任說明']
        elif '學校' in row['學經歷及目前兼任說明']:
            if '教授' in row['學經歷及目前兼任說明']:
                dftemp['經歷'] += row['學經歷及目前兼任說明'] + '、'
            else:
                dftemp['學歷'] += row['學經歷及目前兼任說明']
        elif '商專' in row['學經歷及目前兼任說明']:
            if '教授' in row['學經歷及目前兼任說明']:
                dftemp['經歷'] += row['學經歷及目前兼任說明'] + '、'
            else:
                dftemp['學歷'] += row['學經歷及目前兼任說明']
        ### 可以再額外添加偵測的關鍵字 ###
        
        ### 在這裡以上 ###
        elif row['行次'][0] == 'A': #其他的部分則按照 A=經歷，B=目前兼任的原則填入
            dftemp['經歷'] += row['學經歷及目前兼任說明'] + '、'
        elif row['行次'][0]  == 'B':
            dftemp['目前兼任'] += row['學經歷及目前兼任說明'] + '、'
    
    df_final = df_final.append(dftemp)

In [8]:
df_tobewrite = (df_final.sort_values(by='公司代碼')).reset_index().drop(['index'], axis=1) #重設index

In [9]:
df_tobewrite = df_tobewrite[['公司代碼', '董監經理人姓名', '初任日期--經理人', '初次選任日期--董監事', '姓名代碼', '學歷', '年資--經理人', '年資--董監事', '教育程度', '目前兼任', '經歷', '職位代碼', '職稱', '身份別', '選就任日', '類別代碼']]
#調整欄位順序

In [10]:
df_tobewrite.head()

,公司代碼,董監經理人姓名,初任日期--經理人,初次選任日期--董監事,姓名代碼,學歷,年資--經理人,年資--董監事,教育程度,目前兼任,經歷,職位代碼,職稱,身份別,選就任日,類別代碼
0,2801 彰銀,劉建杉,2018/03,-,P000265011,國立中正大學財務金融研究所,-0.17,-,碩士,無、,,8,紐約分行經理,經理,2018/03/01,3.0
1,2801 彰銀,邱福進,2014/03,-,P000203503,私立中國文化大學企業管理學系,3.83,-,大學,無、,,8,楊梅分行經理,經理,2017/09/01,3.0
2,2801 彰銀,何文科,2014/03,-,P000203502,國立中興大學合作經濟學系,3.83,-,大學,無、,,8,斗南分行經理,經理,2014/03/10,3.0
3,2801 彰銀,沈山鑫,2012/04,-,P000177169,私立致理技術學院財務金融學系,5.75,-,大學,無、,,8,仁愛分行經理,經理,2017/05/01,3.0
4,2801 彰銀,羅順風,2007/04,-,P000149386,國防大學運籌管理研究所,10.75,-,碩士,無、,,8,中壢分行經理,經理,2017/09/01,3.0


In [11]:
college_use = college.set_index('系所')

In [12]:
def schoolparser(string): #清除大學以前的字的函數
    quotation_word = ['大學', '商專', '學院'] #關鍵字們，在這之前的字會被去除掉
    cut = ''
    for word in quotation_word:
        if word in string:
            cut = word
    if cut == '':
        return string
    else:
        stringcut = string.split(cut)
#     print(stringcut[1])
        return stringcut[1]

In [13]:
schoolparser('國立海洋大學會計學系')

'會計學系'

In [14]:
def whichcollege(string):
    department = schoolparser(string) 
    j = 0
    while  j < len(list(college_use.index)) and list(college_use.index)[j] not in department: 
        #檢測專業度資料表裡的科系關鍵字有沒有出現在該學歷裡
        j = j + 1
    if j < len(list(college_use.index)): #有存在則回傳對應的院
        return college_use.loc[list(college_use.index)[j], '院']
    elif j >= len(list(college_use.index)): #不存在則回傳無法辨別
        return('無法辨別')

In [15]:
whichcollege('清華大學不存在學系')

'無法辨別'

In [16]:
whichcollege('清華大學會計學系')

'管理學院'

In [17]:
whichcollege('國立海洋大學企業管理學系')

'管理學院'

In [18]:
df_tobewrite['學歷專業度'] = df_tobewrite['學歷'].map(whichcollege) #填入專業度

In [19]:
df_tobewrite.head()

,公司代碼,董監經理人姓名,初任日期--經理人,初次選任日期--董監事,姓名代碼,學歷,年資--經理人,年資--董監事,教育程度,目前兼任,經歷,職位代碼,職稱,身份別,選就任日,類別代碼,學歷專業度
0,2801 彰銀,劉建杉,2018/03,-,P000265011,國立中正大學財務金融研究所,-0.17,-,碩士,無、,,8,紐約分行經理,經理,2018/03/01,3.0,管理學院
1,2801 彰銀,邱福進,2014/03,-,P000203503,私立中國文化大學企業管理學系,3.83,-,大學,無、,,8,楊梅分行經理,經理,2017/09/01,3.0,管理學院
2,2801 彰銀,何文科,2014/03,-,P000203502,國立中興大學合作經濟學系,3.83,-,大學,無、,,8,斗南分行經理,經理,2014/03/10,3.0,社會科學院
3,2801 彰銀,沈山鑫,2012/04,-,P000177169,私立致理技術學院財務金融學系,5.75,-,大學,無、,,8,仁愛分行經理,經理,2017/05/01,3.0,管理學院
4,2801 彰銀,羅順風,2007/04,-,P000149386,國防大學運籌管理研究所,10.75,-,碩士,無、,,8,中壢分行經理,經理,2017/09/01,3.0,管理學院


In [20]:
df_tobewrite[['公司代碼', '董監經理人姓名', '初任日期--經理人', '初次選任日期--董監事', '姓名代碼', '學歷', '學歷專業度', '年資--經理人', '年資--董監事', '教育程度', '目前兼任', '經歷', '職位代碼', '職稱', '身份別', '選就任日', '類別代碼']].to_excel('test.xlsx', index=False)
#也可以自己改檔名(把test換掉即可)